Импорт библиотек и установка устройства

In [93]:
# Импорт необходимых библиотек
import ffmpeg
import os
import torch
import whisper
import cv2
from PIL import Image
import pandas as pd
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification, 
    MarianMTModel, MarianTokenizer, 
    BartForConditionalGeneration, BartTokenizer, 
    BlipProcessor, BlipForConditionalGeneration
)
import re
import numpy as np
from sklearn.preprocessing import LabelEncoder
import time

# Выбор устройства для вычислений (GPU или CPU)
device = "cuda" if torch.cuda.is_available() else "cpu"


Загрузка моделей и токенизаторов

In [94]:
# Загрузка моделей и токенизаторов
whisper_model = whisper.load_model("small", device=device)  # Модель распознавания речи

translation_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-ru-en").to(device)
translation_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")  # Модель и токенизатор для перевода с русского на английский

back_translation_model = MarianMTModel.from_pretrained("Helsinki-NLP/opus-mt-en-ru").to(device)
back_translation_tokenizer = MarianTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-ru")  # Модель и токенизатор для перевода с английского на русский

summarization_model_name = 'facebook/bart-large-cnn'
summarization_model = BartForConditionalGeneration.from_pretrained(summarization_model_name).to(device)
summarization_tokenizer = BartTokenizer.from_pretrained(summarization_model_name)  # Модель и токенизатор для суммаризации текста


/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Функция для нормализации текста

In [95]:
def normalize_text(text):
    """Нормализация текста: удаление пробелов и нежелательных символов"""
    text = text.strip().lower()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text


Обработка аудио

In [96]:
def process_audio(video_path):
    """Обработка аудио для RuBertAudio: транскрибация, перевод и суммаризация"""
    temp_audio_path = "temp_audio.wav"
    
    try:
        # Извлечение аудио из видео
        ffmpeg.input(video_path, t=90).output(temp_audio_path, acodec='pcm_s16le', ac=1, ar='16000').run(quiet=True)
    except ffmpeg.Error as e:
        print(e.stderr.decode())
        raise

    # Транскрибация аудио
    transcript = whisper_model.transcribe(temp_audio_path)
    os.remove(temp_audio_path)

    # Перевод на английский
    english_text = translator_to_english(transcript['text'])
    # Суммаризация
    summary_english = summarize(english_text)
    # Перевод обратно на русский
    summarized_text = translate_to_russian(summary_english)

    return summarized_text


Функции перевода

In [97]:
def translator_to_english(text):
    """Перевод текста с русского на английский"""
    inputs = translation_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    translated = translation_model.generate(**inputs)
    return translation_tokenizer.decode(translated[0], skip_special_tokens=True)

def translate_to_russian(text):
    """Перевод текста с английского на русский"""
    inputs = back_translation_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
    translated_back = back_translation_model.generate(**inputs)
    return back_translation_tokenizer.decode(translated_back[0], skip_special_tokens=True)


Суммаризация текста

In [98]:
def summarize(text):
    """Суммаризация текста"""
    inputs = summarization_tokenizer(text, return_tensors="pt", max_length=512, truncation=True).to(device)
    summary_ids = summarization_model.generate(inputs['input_ids'], max_length=300, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    return summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)


Обработка видео кадров

In [99]:
def process_video_frames(video_path):
    """Обработка видео кадров и получение текстовых описаний"""
    # Загрузка моделей для генерации описаний
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    caption_model.to(device)

    # Извлечение и обработка кадров видео
    frames = extract_frames(video_path)
    captions = generate_captions(frames, processor, caption_model, device)
    combined_text = ", ".join(captions)

    # Суммаризация и перевод описаний
    summarized_text = summarize(combined_text)
    translated_description = translate_to_russian(summarized_text)

    return translated_description


Извлечение и обработка кадров

In [100]:
def extract_frames(video_path, num_frames=16):
    """Извлечение кадров из видео"""
    vidcap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = max(total_frames // num_frames, 1)
    success, image = vidcap.read()
    count = 0
    while success and len(frames) < num_frames:
        if count % frame_interval == 0:
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            frames.append(Image.fromarray(image_rgb))
        success, image = vidcap.read()
        count += 1
    vidcap.release()
    return frames

def generate_captions(frames, processor, model, device):
    """Генерация описаний для каждого из кадров"""
    captions = []
    unique_captions = set()
    inputs = processor(images=frames, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=30, num_beams=5, early_stopping=True)
    for output in outputs:
        caption = processor.decode(output, skip_special_tokens=True)
        if caption not in unique_captions:
            unique_captions.add(caption)
            captions.append(caption)
    return captions


Предсказание тегов

In [101]:
def predict_tags(model_path, input_text, top_n=3, min_confidence=0.1):
    """Предсказание тегов с выводом самых уверенных, выше порога min_confidence"""
    # Загрузка модели и токенизатора
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model.to(device)

    # Нормализация и токенизация текста
    normalized_text = normalize_text(input_text)  # Убедитесь, что эта функция определена
    encoding = tokenizer(normalized_text, return_tensors='pt', truncation=True, padding=True, max_length=256)
    encoding = {k: v.to(device) for k, v in encoding.items()}

    # Предсказание
    with torch.no_grad():
        outputs = model(**encoding)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    # Загрузка кодировщика меток
    label_encoder = LabelEncoder()
    label_encoder.classes_ = np.load(os.path.join(model_path, 'label_encoder_classes.npy'), allow_pickle=True)

    # Получение топовых тегов, выше порога вероятности
    top_probs, top_indices = torch.topk(probabilities, top_n, dim=1)
    tags_with_probs = [(label_encoder.inverse_transform([idx.item()])[0], prob.item()) 
                       for prob, idx in zip(top_probs[0], top_indices[0]) if prob.item() > min_confidence]

    return tags_with_probs

In [102]:
from collections import defaultdict

def combine_and_select_top_tags(text_tags, audio_tags, video_tags, top_n=5):
    """Объединение и выбор самых уверенных тегов от трех различных моделей, без повторений"""
    
    # Объединение всех тегов в один список
    all_tags = text_tags + audio_tags + video_tags
    
    # Используем словарь для хранения максимальной уверенности для каждого тега
    tag_confidence = defaultdict(lambda: 0)

    # Обновляем словарь, сохраняя максимальную уверенность для каждого тега
    for tag, accuracy in all_tags:
        if accuracy > tag_confidence[tag]:
            tag_confidence[tag] = accuracy

    # Преобразуем в список и сортируем по убыванию уверенности
    sorted_tags = sorted(tag_confidence.items(), key=lambda item: item[1], reverse=True)
    
    # Выбираем топ-N наиболее уверенных тегов
    most_confident_tags = sorted_tags[:top_n]

    # Выводим наиболее уверенные теги и их точность
    print("Наиболее уверенные теги и их точность:")
    for tag, accuracy in most_confident_tags:
        print(f"{tag}: {accuracy:.2f}")

    return most_confident_tags


In [107]:
import os
import pandas as pd
import time

def process_csv(input_csv_path, output_csv_path, videos_path, use_text=True, use_audio=True, use_video=True):
    """Обработка CSV файла и запись обновленного файла с предсказанными тегами"""

    # Чтение CSV файла
    df = pd.read_csv(input_csv_path)
    if os.path.exists(output_csv_path):
        processed_df = pd.read_csv(output_csv_path)
        processed_videos = set(processed_df['video_id'])
    else:
        processed_videos = set()

    total_videos = len(df)

    # Итерация по каждой строке CSV файла
    for index, row in df.iterrows():
        video_id = row['video_id']
        
        if video_id in processed_videos:
            print(f"Video {video_id} already processed. Skipping...")
            continue
        
        title = row['title']
        description = row['description']
        
        # Создание пути к видео файлу
        video_file_path = os.path.join(videos_path, f"{video_id}.mp4")
        print(f"Processing video {index + 1}/{total_videos}: {video_file_path}")

        start_time = time.time()

        # Определяем списки тегов для каждой модели
        text_tags, audio_tags, video_tags = [], [], []

        # Запуск анализа текста
        if use_text:
            combined_text = f"{title} {description}"
            text_tags = predict_tags("RuBertText/saved_model", combined_text)

        # Запуск анализа аудио
        if use_audio:
            summarized_audio_text = process_audio(video_file_path)
            audio_tags = predict_tags("RuBertAudio/saved_model", summarized_audio_text)

        # Запуск анализа видео
        if use_video:
            translated_video_description = process_video_frames(video_file_path)
            video_tags = predict_tags("RuBertVideo/saved_model", translated_video_description)

        # Объединение и выбор самых уверенных тегов
        most_confident_tags = combine_and_select_top_tags(text_tags, audio_tags, video_tags, top_n=3)
        
        # Преобразование тегов в формат кортежей
        predicted_tags = [(str(tag),) for tag, _ in most_confident_tags]

        # Подготовка результата для добавления
        result = {'video_id': video_id, 'predicted_tags': predicted_tags}

        # Открытие и добавление результата в CSV файл
        with open(output_csv_path, 'a') as f:
            result_df = pd.DataFrame([result])
            result_df.to_csv(f, header=f.tell()==0, index=False)

        end_time = time.time()
        print(f"Processed video {video_id} in {end_time - start_time:.2f} seconds. Remaining: {total_videos - (index + 1)} videos.")


Основная логика выполнения

In [108]:
def main(video_path, title, description):
    """Основная функция для обработки видео и получения тегов.
       Возвращает текстовые, аудио и видео теги."""
    
    # Обработка текста
    print("Processing text...")
    combined_text = f"{title} {description}"
    text_tags = predict_tags("RuBertText/saved_model", combined_text)
    print("Text model tags:", text_tags)

    # Обработка аудио
    print("Processing audio...")
    summarized_audio_text = process_audio(video_path)
    audio_tags = predict_tags("RuBertAudio/saved_model", summarized_audio_text)
    print("Audio model tags:", audio_tags)

    # Обработка видео
    print("Processing video frames...")
    translated_video_description = process_video_frames(video_path)
    video_tags = predict_tags("RuBertVideo/saved_model", translated_video_description)
    print("Video model tags:", video_tags)

    # Возврат тегов
    return text_tags, audio_tags, video_tags


Запуск основного метода

In [110]:
# Пример использования
input_csv_path = "/home/user1/hak/video/test_tag_video (2)/sample_submission.csv"
output_csv_path = "/home/user1/hak/video/test_tag_video (2)/updated_submission.csv"
videos_path = "/home/user1/hak/video/test_tag_video (2)/videos"

process_csv(input_csv_path, output_csv_path, videos_path, use_text=True, use_audio=False, use_video=False)

Processing video 1/199: /home/user1/hak/video/test_tag_video (2)/videos/c00d1b173ed8680c155b773d43af7791.mp4
Наиболее уверенные теги и их точность:
Массовая культура: Юмор и сатира: 0.91
Processed video c00d1b173ed8680c155b773d43af7791 in 0.47 seconds. Remaining: 198 videos.
Processing video 2/199: /home/user1/hak/video/test_tag_video (2)/videos/6064c4511a067f2e1d1cef81c21421e8.mp4
Наиболее уверенные теги и их точность:
Массовая культура: 0.64
Processed video 6064c4511a067f2e1d1cef81c21421e8 in 0.45 seconds. Remaining: 197 videos.
Processing video 3/199: /home/user1/hak/video/test_tag_video (2)/videos/3071e2054a02a03150463e514f05fe90.mp4
Наиболее уверенные теги и их точность:
Религия и духовность: астрология: 0.76
Processed video 3071e2054a02a03150463e514f05fe90 in 0.45 seconds. Remaining: 196 videos.
Processing video 4/199: /home/user1/hak/video/test_tag_video (2)/videos/60867179613376c8b56f0705114a7685.mp4
Наиболее уверенные теги и их точность:
Путешествия: 0.26
Транспорт: 0.13
Proce